[View in Colaboratory](https://colab.research.google.com/github/sabumjung/Tensorflow/blob/master/01_simple_policy.ipynb)

In [0]:
#필요한 라이브러리를 임포트하기
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [0]:
#bandit arms 생성하기
#현재 arm4가 가장 큰 reward를 제공한다.
bandit_arms = [0.2,0,-0.2,-2]
num_arms = len(bandit_arms)

def pullBandit(bandit):
    #랜덤 숫자를 얻어오기
    result = np.random.randn(1)
    if result > bandit:
        #결과가 parameter인 bandit값보다 크면 값 1을 리턴함(돈을 얻는 경우)
        return 1
    else:
        #결과가 parameter인 bandit값보다 작으면 값 -1을 리턴함(돈을 잃는 경우)
        return -1

In [0]:
tf.reset_default_graph()

#네트워크의 피드포워드 부분을 설정한다. 
#모든 원소의 값이 1인 텐서를 생성한다. => [1,1,1,1]
weights = tf.Variable(tf.ones([num_arms]))
#activation 함수(소프트맥스)값을 계산한다.
output = tf.nn.softmax(weights)

#다음 6행은 훈련 과정을 설정한다. 
#네트워크 상에서 선택한 행동과 보상을 피드한다.
#보상과 액션정보를 저장한다.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)

#loss를 계산하고 이를 이용하여 네트워크를 업데이트한다.
responsible_output = tf.slice(output,action_holder,[1])
#loss = -log(pi) * A
#Action이 어떤 기준선보다 얼마나 더 나은지의 정도를 의미함
loss = -(tf.log(responsible_output)*reward_holder)

#adamoptimizer를 이용하여 최적화를 수행한다.
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

In [4]:
total_episodes = 1000 #에이전트를 훈련시키기 위한 에피소트의 총 회수를 설정한다.
total_reward = np.zeros(num_arms) #bandit arms의 scoreboard를 0으로 설정한다.


#변수값을 초기화한다.
init = tf.global_variables_initializer()

# 텐서플로 그래프를 런칭한다.
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:     
        #볼츠만 분포에 근거한 action을 선택한다.
        actions = sess.run(output)
        a = np.random.choice(actions,p=actions)
        action = np.argmax(actions == a)

        reward = pullBandit(bandit_arms[action]) #bandit arms중 하나를 선택하여 보상을 계산한다..
        
        #네트워크를 갱신한다.
        _,resp,ww = sess.run([update,responsible_output,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #스코어의 합을 업데이트 한다.
        total_reward[action] += reward
        
        #매 50회 마다 진행결과를 출력한다.
        if i % 50 == 0:
            #4개의 bandit arm에 대한 총 보상을 출력한다.
            print("Running reward for the " + str(num_arms) + " arms of the bandit: " + str(total_reward))
        i+=1
print("\nThe agent thinks arm " + str(np.argmax(ww)+1) + " is the most promising....")

if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 arms of the bandit: [-1.  0.  0.  0.]
Running reward for the 4 arms of the bandit: [-5. -2.  1. 19.]
Running reward for the 4 arms of the bandit: [-9.  0.  3. 25.]
Running reward for the 4 arms of the bandit: [-11.   5.  14.  35.]
Running reward for the 4 arms of the bandit: [-17.   5.  10.  47.]
Running reward for the 4 arms of the bandit: [-17.   3.  14.  69.]
Running reward for the 4 arms of the bandit: [-24.   0.  16.  83.]
Running reward for the 4 arms of the bandit: [-25.  -2.  20.  98.]
Running reward for the 4 arms of the bandit: [-30.  -3.  20. 108.]
Running reward for the 4 arms of the bandit: [-30.  -3.  13. 129.]
Running reward for the 4 arms of the bandit: [-30.  -5.  11. 143.]
Running reward for the 4 arms of the bandit: [-34.  -8.  14. 159.]
Running reward for the 4 arms of the bandit: [-34.  -7.  14. 170.]
Running reward for the 4 arms of the bandit: [-36. -10.  18. 185.]
Running reward for the 4 arms of the bandit: [-41. -10.  18. 198.]
Running